In [2]:
from portfolio_optimization_class import PortfolioOptimization, optimize_windows
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
number_of_quarters = 32
tau = 0.05
maximum_weight = 0.3
data_path = r'data\random_data\n_stocks_per_sector.csv'
portfolios = optimize_windows(number_of_quarters, tau, maximum_weight, data_path)

In [ ]:
copulas = ["clayton_random", "gaussian", "t_student"]
distributions = ["gauss_dist", "t_dist"]

for copula in copulas:
    for window in range(0, 64):
        for dist in distributions:
            if window % 5:
                print(f"Copula: {copula}, Window: {window}, Distribution: {dist}")
            simulated_returns_file_path = r'../data_return_rates/simulated_' + copula + '_RETURNS_22_stocks' + f'_{window}_window_{dist}.csv'
            returns = pd.read_csv(simulated_returns_file_path)

            weights = portfolios.iloc[window].filter(regex="^w\d+$").values
            expectile = -portfolios.iloc[window]["EVAR"]
            weighted_returns = returns.values * weights

            validation_values = (1 - tau) * np.minimum(weighted_returns - expectile, 0) - tau * np.maximum(weighted_returns - expectile, 0)
            scoring_values = (1 - tau) * np.minimum((weighted_returns - expectile)**2, 0) + tau * np.maximum((weighted_returns - expectile)**2, 0)

            expected_validation_values = np.mean(validation_values, axis=1)
            expected_scoring_values = np.mean(scoring_values, axis=1)

            validation_df = pd.DataFrame({
                'Portfolio Return': np.sum(weighted_returns, axis=1),
                'Expected Validation Value': expected_validation_values,
                'Expected Scoring Value': expected_scoring_values
            })
            output_path = r"../scores/" + f'{copula}_22_stocks_{window}_window_{dist}_SCORES.csv'
            validation_df.to_csv(output_path)


In [51]:
# portfolios.iloc[63]

In [ ]:
# returns_df = pd.read_csv(r'../data_return_rates/simulated_clayton_random_RETURNS_22_stocks_0_window_gauss_dist.csv', index_col=0)

# weights = portfolios.iloc[0].filter(regex="^w\d+$").values
# expectile = -portfolios.iloc[0]["EVAR"]


# weighted_returns = returns_df.values * weights


# validation_values = (1 - tau) * np.minimum(weighted_returns - expectile, 0) - tau * np.maximum(weighted_returns - expectile, 0)
# scoring_values = (1 - tau) * np.minimum((weighted_returns - expectile)**2, 0) + tau * np.maximum((weighted_returns - expectile)**2, 0)


# expected_validation_values = np.mean(validation_values, axis=1)
# expected_scoring_values = np.mean(scoring_values, axis=1)


# validation_df = pd.DataFrame({
#     'Portfolio Return': np.sum(weighted_returns, axis=1),
#     'Expected Validation Value': expected_validation_values,
#     'Expected Scoring Value': expected_scoring_values
# })



In [52]:
# validation_df

In [33]:
# import random

# for i in range(200):
#     returns = random.sample(validation_df["Portfolio Return"].to_list(), 12)
#     total_return = 1
#     for return_rate in returns:
#         total_return = total_return * (1 + return_rate)
#     print(total_return)